In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.2 MB/s eta 0:00:00


In [ ]:
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

from sklearn.metrics import f1_score
from torch import nn
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch.nn.functional as F
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import scipy
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from torch.utils.data import Dataset, DataLoader

In [ ]:
dataset = pd.read_csv('dataset_all_nlp_features_target_classes_no_naive (1).csv', sep='\t')
dataset = dataset[dataset.question_id != '129_Чтение текста - видео']
dataset = dataset[dataset['N_words'] > 2]
# dataset = dataset[dataset["verb"] > 0]
# dataset = dataset[dataset["raven"] > 0]

In [ ]:
dataset.describe()

,task_id,user_id,age,E,A,C,N,O,AP,AM,...,raven_classes,verb_classes,AM_classes,AP_classes,BM_classes,BP_classes,GM_classes,GP_classes,DM_classes,DP_classes
count,6477.000000,6477.000000,1736.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,...,6467.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000
mean,6.098966,658.156091,23.868664,3.520113,3.681306,3.681509,2.960723,3.755056,3.840862,2.150763,...,1.666306,2.035510,0.516134,1.232978,0.844527,0.920179,0.710514,1.008492,0.695538,1.330400
std,3.536441,279.490692,6.199752,0.827778,0.678065,0.826531,0.894088,0.653033,0.551309,0.681014,...,1.077559,1.333504,0.725172,0.762120,0.810527,0.759715,0.781998,0.772894,0.826484,0.725106
min,1.000000,124.000000,18.000000,1.130000,1.444444,1.330000,1.000000,1.700000,1.670000,1.000000,...,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,437.000000,20.000000,2.880000,3.222222,3.000000,2.250000,3.300000,3.555556,1.666667,...,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,6.000000,654.000000,22.000000,3.500000,3.670000,3.777778,3.000000,3.800000,3.888889,2.110000,...,2.000000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,9.000000,874.000000,25.000000,4.250000,4.220000,4.333333,3.625000,4.300000,4.220000,2.555556,...,2.000000,3.000000,1.000000,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000
max,12.000000,1167.000000,51.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,4.000000,4.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [ ]:
def get_circumplex_labels(dataset):
    AP_LABELS = list(set(dataset['AP_classes'].unique()))
    AP_SCORE_INDICES = range(0, 3)
    AM_LABELS = list(set(dataset['AM_classes'].unique()))
    AM_SCORE_INDICES = range(3, 6)

    BP_LABELS = list(set(dataset['BP_classes'].unique()))
    BP_SCORE_INDICES = range(6, 9)
    BM_LABELS = list(set(dataset['BM_classes'].unique()))
    BM_SCORE_INDICES = range(9, 12)

    GP_LABELS = list(set(dataset['GP_classes'].unique()))
    GP_SCORE_INDICES = range(12, 15)
    GM_LABELS = list(set(dataset['GM_classes'].unique()))
    GM_SCORE_INDICES = range(15, 18)

    DP_LABELS = list(set(dataset['DP_classes'].unique()))
    DP_SCORE_INDICES = range(18, 21)
    DM_LABELS = list(set(dataset['DM_classes'].unique()))
    DM_SCORE_INDICES = range(21, 24)

    CIRCLE_LABELS = AP_LABELS + AM_LABELS + BP_LABELS + BM_LABELS + GP_LABELS + GM_LABELS + DP_LABELS + DM_LABELS
    INDEXEX_CIRCLE = [AP_SCORE_INDICES, AM_SCORE_INDICES, BP_SCORE_INDICES,
                      BM_SCORE_INDICES, GP_SCORE_INDICES, GM_SCORE_INDICES,
                      DP_SCORE_INDICES, DM_SCORE_INDICES]
    return INDEXEX_CIRCLE, CIRCLE_LABELS


In [ ]:
curc_indexes, curc_labels = get_circumplex_labels(dataset)

In [ ]:
curc_indexes

[range(0, 3),
 range(3, 6),
 range(6, 9),
 range(9, 12),
 range(12, 15),
 range(15, 18),
 range(18, 21),
 range(21, 24)]

In [ ]:
class DataPreparation(Dataset):
    
    def __init__(self, tokenizer, data, target_values='circumplex', max_length=None):
        
        self.tokenizer = tokenizer
        self.data = data
        self.t_val = target_values
        
        if max_length == None:
            max_length_counted = data["text"].str.split(' ').str.len().max(axis=0)
            self.max_length = max_length_counted if max_length_counted < 512 else 512
        else:
            self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def tokenize(self, text):

        tokens = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt')

        return tokens
    

    def reorganize_labels(self, row):

        def return_zeros(row_data, classes):
          labels_zeros = np.zeros(len(set(self.data[classes].unique())))
          labels_zeros[row_data[classes]] = 1
          return labels_zeros

        return np.concatenate((return_zeros(row, 'AP_classes'), return_zeros(row, 'AM_classes'),
                               return_zeros(row, 'BP_classes'), return_zeros(row, 'BM_classes'),
                               return_zeros(row, 'GP_classes'), return_zeros(row, 'GM_classes'),
                               return_zeros(row, 'DP_classes'), return_zeros(row, 'DM_classes')
                               ))  


    def __getitem__(self, index):
        
        source_text = self.data['text'].iloc[index]
        source = self.tokenize(source_text)

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        
        labels = self.reorganize_labels(self.data.iloc[index])
        

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "labels":  labels
        }

In [ ]:
model_name = 'DeepPavlov/rubert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
np.random.seed(112)
train_data, vaild_data, test_data = np.split(dataset.sample(frac=1, random_state=42), 
                                     [int(.80*len(dataset)), int(.91*len(dataset))])

print(len(train_data),len(vaild_data), len(test_data))

5181 713 583


In [ ]:
train_data.head(2)

,text,f,filename,task_id,question_id,user_id,gender,age,E,A,...,raven_classes,verb_classes,AM_classes,AP_classes,BM_classes,BP_classes,GM_classes,GP_classes,DM_classes,DP_classes
2138,стала лучшим консультантом памперс,NaN,аудио/answer11_113_12665_3_241.wav,11,134_Видео-интервью,241,NaN,NaN,4.13,3.330000,...,0.0,0,0,2,2,1,1,1,2,0
2467,Вы знаете я нашла себе работу по любимую работу,NaN,аудио/answer11_113_20670_6_782.wav,11,134_Видео-интервью,782,NaN,NaN,4.00,3.555556,...,4.0,4,1,0,0,0,0,0,0,1


In [ ]:
train_dataset_data = DataPreparation(
    tokenizer=tokenizer,
    data = train_data,
    max_length = 120,
)

val_dataset_data = DataPreparation(
    tokenizer=tokenizer,
    data = vaild_data,
    max_length = 120,
)

test_dataset_data = DataPreparation(
    tokenizer=tokenizer,
    data = test_data,
    max_length = 120,
)

In [ ]:
train_dataset = DataLoader(train_dataset_data, batch_size=8, drop_last=True, shuffle=True)
val_dataset = DataLoader(val_dataset_data, batch_size=8)
test_dataset = DataLoader(test_dataset_data, batch_size=8)

In [ ]:
b = next(iter(train_dataset))
for k, v in b.items():
    print(f'{k} shape: {v.shape}')
    # print(b['labels'])

source_ids shape: torch.Size([8, 120])
source_mask shape: torch.Size([8, 120])
labels shape: torch.Size([8, 24])


In [ ]:
class BertBaseline(nn.Module):
    
    def __init__(self, model_name, out_features, inner_feautes=256):
        super(BertBaseline, self).__init__()

        self.bert = AutoModel.from_pretrained(model_name, return_dict=True)

        self.linear_modules = nn.ModuleList([torch.nn.Linear(self.bert.config.hidden_size, inner_feautes),
                                          torch.nn.Linear(inner_feautes, inner_feautes),
                                          torch.nn.Linear(inner_feautes, out_features)])
        
        self.dropout = nn.Dropout(0.4)
        self.layer_norm = nn.LayerNorm(inner_feautes)
        self.relu = nn.ReLU()   
    
    def forward(self, input_ids, attention_mask):

         bert = self.bert(
            input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True
            )
#          x = bert[0][:, 0]  # last hidden state output
         token_embeddings = bert[0]
         input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
         sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
         sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-6)
         x = sum_embeddings / sum_mask
        
         h = []
        
         for lin in self.linear_modules[:-1]:
            x = lin(x)
            h.append(x)
            x = self.layer_norm(x)
            x = self.relu(x)
            x = self.dropout(x)
        
#          x = self.relu(x)
         x = self.relu(h[-1] + h[-2])
         x = self.linear_modules[-1](x)
        
         return x

In [ ]:
model = BertBaseline(model_name, inner_feautes=512, out_features=len(curc_labels))

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()
epochs = 15
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

Надо сделать прогон на каждом классе отдельно, чтобы определеить, какие будут веса можно будет использовать для общего лосса

In [ ]:
def analysing_logits(logits, class_indexes):
  
  def compute_maxes(logits, CUR_SCORE_INDEX):
    labels_zeros = np.zeros(logits[:, CUR_SCORE_INDEX].shape)
    pred_class = np.argmax(logits[:, CUR_SCORE_INDEX], axis=1)
    labels_zeros[list(range(len(labels_zeros))), pred_class] = 1
    return labels_zeros

  list_of_best_labels = []

  for circ_cl in class_indexes:
    labels_zeros = compute_maxes(logits, circ_cl)
    list_of_best_labels.append(labels_zeros)
  
  return np.concatenate((list_of_best_labels), axis=1)

In [ ]:
def init_adding_weights():
    return (1/1.09, 1/1.09, 1/1.09, 1/1.09,
            1/1.09, 1/1.09, 1/1.09, 1/1.09)

In [ ]:
# for ind, circ_cl in enumerate(curc_indexes):
#       print('________', ind, circ_cl)

In [ ]:
def comupte_custom_loss(labels, logits, criterion, class_indexes):
    

    custom_weights = init_adding_weights()

    custom_loss = 0
    for ind, circ_cl in enumerate(class_indexes):
      custom_loss += custom_weights[ind] * criterion(logits[:, circ_cl],
                                  labels[:, circ_cl])

    return custom_loss

In [ ]:
def show_metrics(labels, predicted, class_indexes):

    labels_names = ['AP', 'AM', 'BP', 'BM',
              'GP', 'GM', 'DP', 'DM']
    
    labels = np.vstack(labels)
    predicted = np.vstack(predicted)

    for lab, circ_cl in zip(labels_names, class_indexes):
      
      f1_sc = f1_score(labels[:, circ_cl],
               predicted[:, circ_cl], average="macro")
      accur = accuracy_score(labels[:, circ_cl],
               predicted[:, circ_cl])
      
      
      print(f'Scores for {lab}\n==================\n')
      print('Macro F1 score for {0} - {1}\n'.format(lab, f1_sc))
      print('Accuracy score for {0} - {1}\n'.format(lab, accur))
      print(classification_report(labels[:, circ_cl], predicted[:, circ_cl], zero_division=0))


In [ ]:
def train(model, data_loader, device, optimizer, criterion, n_epoch, class_indexes):

    print('Epoch #{}\n'.format(n_epoch+1))

    train_losses = []
    train_labels = []
    train_predictions = []   

    progress_bar = tqdm(total=int(len(data_loader.dataset)/data_loader.batch_size), 
                        desc='Epoch {}'.format(n_epoch + 1))

    model.train()

    for _, data in enumerate(data_loader, 0):


          input_ids = data["source_ids"].to(device)
          attention_mask = data["source_mask"].to(device)
          labels = data['labels'].to(device)

          optimizer.zero_grad()

          pred = model(input_ids=input_ids, attention_mask=attention_mask)

          loss = comupte_custom_loss(labels, pred, criterion, class_indexes)
              
          loss.backward()
              
          optimizer.step()

          resulting_output = analysing_logits(pred.detach().cpu().numpy(), class_indexes)

          train_losses.append(loss.item())
          train_labels.extend(labels.cpu().detach().numpy())
          train_predictions.extend(resulting_output)

          progress_bar.set_postfix(loss=np.mean(train_losses))
          progress_bar.update(1)

    progress_bar.close()
  
    
    print('\n\nMean Loss after epoch #{0} - {1}'.format(str(n_epoch + 1), np.mean(train_losses)))
    print(f'\n Scores after {n_epoch + 1} on train:')
    show_metrics(train_labels, train_predictions, class_indexes)

In [ ]:
def validating(model, data_loader, device, optimizer, criterion, n_epoch, class_indexes):

    val_losses, val_labels, val_predictions = [], [], []

    progress_bar = tqdm(total=int(len(data_loader.dataset)/data_loader.batch_size),
                        desc='Epoch {}'.format(n_epoch + 1))

    model.eval()

    for _, data in enumerate(data_loader, 0):
          
          input_ids = data["source_ids"].to(device)
          attention_mask = data["source_mask"].to(device)
          labels = data['labels'].to(device)

          with torch.no_grad():
              pred = model(input_ids, attention_mask)

          loss = comupte_custom_loss(labels, pred, criterion, class_indexes)
          
          resulting_output = analysing_logits(pred.detach().cpu().numpy(), class_indexes)

          val_losses.append(loss.item())
          val_labels.extend(labels.cpu().detach().numpy())
          val_predictions.extend(resulting_output)

          progress_bar.set_postfix(loss=np.mean(val_losses))
          progress_bar.update(1)

    progress_bar.close()
    
    
    valid_stats.append(
        {
            'Val Loss': np.mean(val_losses)
        }
    )

    print('\n\nMean Loss after epoch #{0} - {1}'.format(str(n_epoch + 1), np.mean(val_losses)))
    print(f'\n Scores after {n_epoch + 1} on validation:')
    show_metrics(val_labels, val_predictions, class_indexes)
    
    return valid_stats

In [ ]:
import os

In [ ]:
global valid_stats
valid_stats = []
best_valid_loss = float('inf')

# for each epoch
for epoch in range(epochs):
    # train

    try:
        train(model, train_dataset, device, optimizer, criterion, epoch, curc_indexes)
          # # validate
        
        valid_stats = validating(model, val_dataset, device, optimizer, criterion, epoch, curc_indexes )

        if valid_stats[epoch]['Val Loss'] < best_valid_loss:
            print(f"Best validation loss - {valid_stats[epoch]['Val Loss']}")
            best_valid_loss = valid_stats[epoch]['Val Loss']
            
            name_to_save = f'model_baseline_basic_all_circ'
            if os.path.isfile('results/'+name_to_save+'.pth'):
                    os.remove('results/'+name_to_save+'.pth')
                    torch.save(model.state_dict(), 'results/'+name_to_save+'.pth')
            else:
                    if not os.path.isdir('results'):
                        os.mkdir('results')
                    torch.save(model.state_dict(), 'results/'+name_to_save+'.pth')
#                     else:
#                         os.mkdir('results')
    except KeyboardInterrupt:
        break

Epoch #1



Epoch 1:   0%|          | 0/647 [00:00<?, ?it/s]



Mean Loss after epoch #1 - 7.568306777760288

 Scores after 1 on train:
Scores for AP

Macro F1 score for AP - 0.33579686922448504

Accuracy score for AP - 0.4476429675425039

              precision    recall  f1-score   support

           0       0.35      0.03      0.05      1055
           1       0.41      0.37      0.39      1895
           2       0.47      0.71      0.56      2226

   micro avg       0.45      0.45      0.45      5176
   macro avg       0.41      0.37      0.34      5176
weighted avg       0.42      0.45      0.40      5176
 samples avg       0.45      0.45      0.45      5176

Scores for AM

Macro F1 score for AM - 0.25986573432577614

Accuracy score for AM - 0.6108964451313755

              precision    recall  f1-score   support

           0       0.62      0.99      0.76      3186
           1       0.20      0.00      0.00      1267
           2       0.15      0.01      0.02       723

   micro avg       0.61      0.61      0.61      5176
   macro av

Epoch 1:   0%|          | 0/89 [00:00<?, ?it/s]



Mean Loss after epoch #1 - 7.379049104443778

 Scores after 1 on validation:
Scores for AP

Macro F1 score for AP - 0.37233994820749233

Accuracy score for AP - 0.4796633941093969

              precision    recall  f1-score   support

           0       0.36      0.04      0.07       131
           1       0.41      0.58      0.48       253
           2       0.56      0.58      0.57       329

   micro avg       0.48      0.48      0.48       713
   macro avg       0.44      0.40      0.37       713
weighted avg       0.47      0.48      0.45       713
 samples avg       0.48      0.48      0.48       713

Scores for AM

Macro F1 score for AM - 0.2628150665533843

Accuracy score for AM - 0.6507713884992987

              precision    recall  f1-score   support

           0       0.65      1.00      0.79       464
           1       0.00      0.00      0.00       166
           2       0.00      0.00      0.00        83

   micro avg       0.65      0.65      0.65       713
   macr

...

Epoch 8:   0%|          | 0/647 [00:00<?, ?it/s]



Mean Loss after epoch #8 - 4.5885838046867296

 Scores after 8 on train:
Scores for AP

Macro F1 score for AP - 0.7246463433407798

Accuracy score for AP - 0.731646058732612

              precision    recall  f1-score   support

           0       0.73      0.68      0.70      1057
           1       0.64      0.70      0.67      1894
           2       0.82      0.79      0.80      2225

   micro avg       0.73      0.73      0.73      5176
   macro avg       0.73      0.72      0.72      5176
weighted avg       0.74      0.73      0.73      5176
 samples avg       0.73      0.73      0.73      5176

Scores for AM

Macro F1 score for AM - 0.7103326899366434

Accuracy score for AM - 0.776468315301391

              precision    recall  f1-score   support

           0       0.83      0.91      0.87      3186
           1       0.60      0.45      0.51      1267
           2       0.74      0.76      0.75       723

   micro avg       0.78      0.78      0.78      5176
   macro avg  

Epoch 8:   0%|          | 0/89 [00:00<?, ?it/s]



Mean Loss after epoch #8 - 9.144512401283414

 Scores after 8 on validation:
Scores for AP

Macro F1 score for AP - 0.3964851468281591

Accuracy score for AP - 0.4628330995792426

              precision    recall  f1-score   support

           0       0.41      0.12      0.19       131
           1       0.39      0.60      0.47       253
           2       0.57      0.50      0.53       329

   micro avg       0.46      0.46      0.46       713
   macro avg       0.46      0.40      0.40       713
weighted avg       0.48      0.46      0.45       713
 samples avg       0.46      0.46      0.46       713

Scores for AM

Macro F1 score for AM - 0.38172064968706865

Accuracy score for AM - 0.6100981767180925

              precision    recall  f1-score   support

           0       0.68      0.84      0.76       464
           1       0.29      0.22      0.25       166
           2       0.41      0.08      0.14        83

   micro avg       0.61      0.61      0.61       713
   macr

Epoch 9:   0%|          | 0/647 [00:00<?, ?it/s]

In [ ]:
def test(data_loader, device, class_indexes):

    val_losses, val_labels, val_predictions = [], [], []
    model.load_state_dict(torch.load('results/model_baseline_basic_all_circ.pth'))
    

    model.eval()

    for _, data in enumerate(data_loader, 0):
          
          input_ids = data["source_ids"].to(device)
          attention_mask = data["source_mask"].to(device)
          labels = data['labels'].to(device)

          with torch.no_grad():
              pred = model(input_ids, attention_mask)

          
          resulting_output = analysing_logits(pred.detach().cpu().numpy(), class_indexes)

          val_labels.extend(labels.cpu().detach().numpy())
          val_predictions.extend(resulting_output)



    show_metrics(val_labels, val_predictions, class_indexes)
    # print('F1 score after epoch #{0} on validation - {1}\n'.format(str(n_epoch + 1), f1_score(val_labels, val_predictions, average='macro'))

In [ ]:
test(train_dataset, device, curc_indexes)

Scores for AP

Macro F1 score for AP - 0.5949840041984099

Accuracy score for AP - 0.6147604327666152

              precision    recall  f1-score   support

           0       0.66      0.42      0.51      1057
           1       0.51      0.66      0.58      1897
           2       0.73      0.67      0.70      2222

   micro avg       0.61      0.61      0.61      5176
   macro avg       0.63      0.58      0.59      5176
weighted avg       0.63      0.61      0.61      5176
 samples avg       0.61      0.61      0.61      5176

Scores for AM

Macro F1 score for AM - 0.40085333665361306

Accuracy score for AM - 0.6501159196290572

              precision    recall  f1-score   support

           0       0.65      0.98      0.79      3184
           1       1.00      0.00      0.01      1268
           2       0.59      0.31      0.41       724

   micro avg       0.65      0.65      0.65      5176
   macro avg       0.75      0.43      0.40      5176
weighted avg       0.73      0.6

In [ ]:
test(val_dataset, device, curc_indexes)

Scores for AP

Macro F1 score for AP - 0.4262570652018424

Accuracy score for AP - 0.4782608695652174

              precision    recall  f1-score   support

           0       0.40      0.18      0.25       131
           1       0.41      0.60      0.48       253
           2       0.59      0.50      0.54       329

   micro avg       0.48      0.48      0.48       713
   macro avg       0.47      0.43      0.43       713
weighted avg       0.49      0.48      0.47       713
 samples avg       0.48      0.48      0.48       713

Scores for AM

Macro F1 score for AM - 0.32079450561650646

Accuracy score for AM - 0.6479663394109397

              precision    recall  f1-score   support

           0       0.66      0.97      0.79       464
           1       1.00      0.01      0.01       166
           2       0.33      0.11      0.16        83

   micro avg       0.65      0.65      0.65       713
   macro avg       0.66      0.36      0.32       713
weighted avg       0.70      0.6

In [ ]:
test(test_dataset, device, curc_indexes)

Scores for AP

Macro F1 score for AP - 0.43623481781376516

Accuracy score for AP - 0.4854202401372213

              precision    recall  f1-score   support

           0       0.41      0.19      0.26       114
           1       0.42      0.57      0.49       214
           2       0.58      0.54      0.56       255

   micro avg       0.49      0.49      0.49       583
   macro avg       0.47      0.44      0.44       583
weighted avg       0.49      0.49      0.47       583
 samples avg       0.49      0.49      0.49       583

Scores for AM

Macro F1 score for AM - 0.3027432902020361

Accuracy score for AM - 0.6500857632933105

              precision    recall  f1-score   support

           0       0.66      0.99      0.79       376
           1       0.00      0.00      0.00       120
           2       0.43      0.07      0.12        87

   micro avg       0.65      0.65      0.65       583
   macro avg       0.36      0.35      0.30       583
weighted avg       0.49      0.6